# Running germ selection on multiple processors
The code below should be put into a script and run using mpiexec.  It's primary function is to pass a MPI Comm object to `pygsti.algorithms.germselection.build_up_breadth`.

In [1]:
from __future__ import print_function
import time

import pygsti
from pygsti.modelpacks import smq2Q_XYICNOT
from pygsti.algorithms import germselection as germsel

In [1]:
from mpi4py import MPI
comm = MPI.COMM_WORLD

In [1]:
def do_greedy_germsel(target_model, forced_germs, candidate_counts,
                      seedStart, outFilename, comm):
    #candidate_counts is a dict of keys = germ lengths, values = # of germs at that length                                                            

    tStart = time.time()

    candidate_germs = []
    for i,(germLength, count) in enumerate(candidate_counts.items()):
        if count == "all upto":
            candidate_germs.extend( pygsti.circuits.list_all_circuits_without_powers_and_cycles(
                    target_model.operations.keys(), max_length=germLength) )
        else:
            candidate_germs.extend( pygsti.circuits.list_random_circuits_onelen(
                    target_model.operations.keys(), germLength, count, seed=seedStart+i))

    available_germs = pygsti.tools.remove_duplicates( forced_germs + candidate_germs )
    print("%d available germs" % len(available_germs))
    germs = germsel.find_germs_breadthfirst(target_model, available_germs,
                     randomization_strength=1e-3, num_copies=3, seed=1234,
                     op_penalty=10.0, score_func='all', tol=1e-6, threshold=1e5,
                     pretest=False, force=forced_germs, verbosity=5, comm=comm, mem_limit=0.5*(1024**3))

    if comm is None or comm.Get_rank() == 0:
        print("Germs (%d) = \n" % len(germs), "\n".join(map(str,germs)))
        print("Total time = %mdl" % (time.time()-tStart))
        pickle.dump(germs,open(outFilename,"wb"))
    return germs

In [1]:
#2Q case                                                                                                                                              
target_model = smq2Q_XYICNOT.target_model()
forced_germs = pygsti.circuits.to_circuits([(gl,) for gl in target_model.operations.keys()]) #singletons                                                                                      
candidate_counts = { 3:"all upto", 4:30, 5:20, 6:20, 7:20, 8:20} # germLength:num_candidates                                                          
seedStart = 4

In [1]:
do_greedy_germsel(target_model, forced_germs, candidate_counts,
                  seedStart, "germs_EXAMPLE.pkl", comm)

201 available germs
Starting germ set optimization. Lower score is better.
Memory estimate of 13.6 GB (0.5 GB limit) for all-Jac mode.
Memory estimate of 0.2 GB (0.5 GB limit) for single-Jac mode.
    Initial germ set computation Iter 1 of 6 []: 
    Initial germ set computation Iter 2 of 6 Gxpi2:1@(1): 
    Initial germ set computation Iter 3 of 6 Gypi2:1@(1): 
    Initial germ set computation Iter 4 of 6 Gxpi2:0@(0): 
    Initial germ set computation Iter 5 of 6 Gypi2:0@(0): 
    Initial germ set computation Iter 6 of 6 Gcnot:0:1@(0,1): 
  Outer iteration: 1 of 1282 amplified, 6 germs
    Inner iter over candidate germs Iter 001 of 195 []Gxpi2:1@(1): 
      Score: major=-114.0 minor=275.75502325607374, N: 194
    Inner iter over candidate germs Iter 002 of 195 []Gypi2:1@(1): 
      Score: major=-114.0 minor=275.7522442827058, N: 194
    Inner iter over candidate germs Iter 003 of 195 []Gxpi2:0@(0): 
      Score: major=-114.0 minor=303.12985712455327, N: 194
    Inner iter over candid

KeyboardInterrupt: 

Above is **keyboard-interrupted on purpose**, as this output was produced with a single processor and it would have taken a very long time.
